In [17]:
from glob import glob
import pandas as pd
import numpy

In [19]:
file_name = "stocktakes/mb*.csv"
df = pd.DataFrame()
df_list = []
for f in glob(file_name):
    print(f)
    sub_df = pd.read_csv(f)
    df_list.append(sub_df)
df = pd.concat(df_list,axis=0)
    
    
df["size"] = df.SKU.apply(lambda x:"".join(list(x)[-3:]))


def remove_sizes(string):
    string = string.replace("MED","")
    string = string.replace("XXL","")
    string = string.replace("SML","")
    string = string.replace("MED","")
    string = string.replace("XXS","")
    string = string.replace("LRG","")
    string = string.replace("XLG","")
    string = string.replace("XSM","")
    return string
    
    
df["SKU_noSize"] = df.SKU.apply(lambda x:remove_sizes(x))
minus = df.loc[df.Adjustment < 0]
plus = df.loc[df.Adjustment >0]
print(minus.Adjustment.sum())
print(plus.Adjustment.sum())
display(minus.groupby(["SKU_noSize","size"]).agg({"Adjustment":"sum"}))
pd.set_option("display.max_rows", None, "display.max_columns", None)

display(plus.groupby(["SKU_noSize","size"]).agg({"Adjustment":"sum"}))


stocktakes/mb_2.csv
stocktakes/mb_1.csv
-2
3


,,Adjustment
SKU_noSize,size,
V370CCL,SML,-1
V370KHK,LRG,-1


Adjustment
SKU_noSize size            
V370CCL    LRG            1
           MED            1
V434BLK    LRG            1

## plus["SKU_noSize"].nunique()

In [10]:
file_name = "nightly_logs/mb.csv"
delimiter = ","
dates = ["3/25/2021"]

df2 = pd.read_csv(file_name,delimiter=delimiter)
df2.columns = df2.iloc[0]

def apply_numeric_size(string):
    if string.isnumeric():
        string = string.zfill(3)
    return string

df2=df2.dropna(subset=["Date","SKU","Style"])
df2 = df2.loc[df2.Date.isin(dates)]
df2.Size = df2.Size.apply(lambda x:apply_numeric_size(x))
df2["SKU_full"] = df2["SKU"] + df2["Color"] + df2["Size"]
df2["SKU_partial"] = df2["SKU"]+df2["Color"]
df2 = df2.loc[df2["Processed\n(Stocky)"] == "FALSE"]
df2["Quantity"] = df2["Quantity"].apply(lambda x:int(x))
print("Return/Damaged/Giftcard and their count of rows")
print(df2.groupby(["Notes (LightSpeed Return or Damaged)"]).Quantity.sum())
damaged = df2.loc[df2["Notes (LightSpeed Return or Damaged)"].isin(["Damage","Dmage","Damage ","Damaged"])]
print("=========")
print("Damaged:")
damaged.groupby(["SKU_partial","Size"]).agg({"Quantity":"sum"})


Return/Damaged/Giftcard and their count of rows
Notes (LightSpeed Return or Damaged)
Damage               7
Gift Return          1
Gift Return          4
Lightspeed Return    3
Name: Quantity, dtype: int64
Damaged:


,,Quantity
SKU_partial,Size,
V163HTG,L,1
V420INK,036,1
VW133LHG,S,1
VW303BLK,M,1
VW303HTG,XS,1
VW428BLK,S,1


In [11]:
damaged.Quantity.sum()

7

In [16]:
#damaged["SKU_full"] = damaged.SKU_full.apply(lambda x:x.replace(" ",""))
damaged.groupby("SKU_full").agg({"Quantity":"sum"}) * -1

,Quantity
SKU_full,
V163HTGL,-1
V420INK036,-1
VW133LHGS,-1
VW303BLKM,-1
VW303HTGXS,-1
VW428BLKS,-1


In [14]:
lightspeed_return = df2.loc[df2["Notes (LightSpeed Return or Damaged)"]=="Lightspeed Return"]
print(lightspeed_return.Quantity.sum())

display(lightspeed_return.groupby(["SKU_partial","Size"]).agg({"Quantity":"sum"}))
display(lightspeed_return.groupby("SKU_full").agg({"Quantity":"sum"}))


3


,,Quantity
SKU_partial,Size,
V302IND,L,1
V418HTI,L,1
VW401CCL,XS,1


,Quantity
SKU_full,
V302INDL,1
V418HTIL,1
VW401CCLXS,1


In [15]:

giftcard = df2.loc[df2["Notes (LightSpeed Return or Damaged)"].isin(["Lightspeed Return Gift Card","Lightspeed Return Giftcard","Lightspeed Giftcard","Gift Return","Gift Return ","Gift","Gift REturn "])]
print(giftcard.Quantity.sum())

display(giftcard.groupby(["SKU_partial","Size"]).agg({"Quantity":"sum"}))
display(giftcard.groupby("SKU_full").agg({"Quantity":"sum"}))


5


,,Quantity
SKU_partial,Size,
V302CCL,M,1
V416HPT,L,1
V416IND,L,1
VW142HCC,M,1
VW303-LBLK,S,1


,Quantity
SKU_full,
V302CCLM,1
V416HPTL,1
V416INDL,1
VW142HCCM,1
VW303-LBLKS,1
